# RAG with Llama Stack Primitives

This notebook demonstrates building a **Retrieval-Augmented Generation (RAG)** system using **pure Llama Stack primitives** - no frameworks like LangChain or CrewAI.

## Agentic RAG Approach

This notebook shows the **Agentic RAG loop**:

1. **Query** → User asks a question
2. **Search** → Semantic search in vector store
3. **Extract** → Get relevant document chunks
4. **Generate** → Pass context + query to LLM
5. **Answer** → Return generated response

## Architecture

```
┌─────────────────────────────────────────────┐
│         User Query                          │
└────────┬────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────┐
│  Vector Search                              │
│  (client.vector_stores.search)              │
└────────┬────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────┐
│  Context Extraction                         │
│  (Extract relevant chunks)                  │
└────────┬────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────┐
│  Llama Stack Responses API                  │
│  (client.responses.create)                  │
└────────┬────────────────────────────────────┘
         │
         ▼
┌─────────────────────────────────────────────┐
│         Final Answer                        │
└─────────────────────────────────────────────┘
```

## Configuration

This notebook uses environment variables from `.env` file in the project root.
Create your own `.env` file based on `.env.example`.

## Install Required Packages

Install Llama Stack client (version must match server version):

In [ ]:
!pip3 install -q "llama-stack-client==0.3.0" "python-dotenv" "requests"

## Import Dependencies

In [ ]:
# Core imports
import os
import sys
import requests
from pathlib import Path
from io import BytesIO
from dotenv import load_dotenv, find_dotenv

# Llama Stack client
from llama_stack_client import LlamaStackClient

# --- Load environment variables ---
# Automatically detect the nearest .env (walks up from current directory)
env_path = find_dotenv(usecwd=True)
if env_path:
    load_dotenv(env_path)
    print(f"📁 Loading environment from: {env_path}")
    print("✅ .env file FOUND and loaded")
else:
    default_path = Path.cwd() / ".env"
    print(f"📁 No .env found via find_dotenv — checked: {default_path}")
    print("⚠️  .env file NOT FOUND")

# --- Verify Python interpreter / kernel ---
print(f"\n🐍 Python: {sys.executable}")

# Detect if running inside a virtual environment
in_venv = (
    hasattr(sys, "real_prefix") or
    (getattr(sys, "base_prefix", sys.prefix) != sys.prefix) or
    "VIRTUAL_ENV" in os.environ or
    "CONDA_PREFIX" in os.environ
)

if in_venv:
    print("✅ Using virtual environment - CORRECT!")
else:
    print("⚠️  Using global Python - Consider switching kernel!")
    print("   Click 'Select Kernel' → Choose 'Python (byo-agentic-framework)')")

## Configure Llama Stack Connection

Create client using Llama Stack's base URL:

In [ ]:
# Get configuration from environment variables
LLAMA_STACK_BASE_URL = os.getenv("LLAMA_STACK_BASE_URL")
INFERENCE_MODEL = os.getenv("INFERENCE_MODEL")
LLAMA_STACK_OPENAI_ENDPOINT = os.getenv("LLAMA_STACK_OPENAI_ENDPOINT")

print("🌐 Llama Stack Configuration:")
print(f"   Base URL: {LLAMA_STACK_BASE_URL}")
print(f"   Model: {INFERENCE_MODEL}")
print(f"   OpenAI Endpoint: {LLAMA_STACK_OPENAI_ENDPOINT}")

# Create client using the base URL
client = LlamaStackClient(base_url=LLAMA_STACK_BASE_URL)
print("\n✅ Llama Stack client created")

## Vector Store Setup

Create a vector store with Parasol Cloud Corp earnings report.

### Step 1: Download PDF Document

We'll download the FY2025 earnings report from GitHub:

In [ ]:
print("📚 Downloading Parasol Cloud Corp earnings report...\n")

# PDF document URL
pdf_url = "https://raw.githubusercontent.com/rh-ai-quickstart/byo-agentic-framework/dev/assets/images/Parasol_Cloud_Corp_Earnings_Report.pdf"

# Download the PDF
print(f"📥 Downloading from: {pdf_url}")
response = requests.get(pdf_url)
response.raise_for_status()  # Raise error if download fails

pdf_content = response.content
print(f"✅ Downloaded {len(pdf_content)} bytes")
print(f"   Content type: application/pdf")

### Step 2: Upload PDF to Llama Stack

Upload the earnings report PDF using the Files API:

In [ ]:
print("📤 Uploading PDF to Llama Stack...\n")

# Create in-memory file from PDF content
with BytesIO(pdf_content) as pdf_buffer:
    # Set filename for the PDF
    pdf_buffer.name = "Parasol_Cloud_Corp_Earnings_Report.pdf"
    
    # Upload to Llama Stack
    file_response = client.files.create(
        file=pdf_buffer,
        purpose="assistants"
    )
    
    file_id = file_response.id
    print(f"✅ Uploaded: Parasol Cloud Corp FY2025 Earnings Report")
    print(f"   File ID: {file_id}")
    print(f"   Filename: Parasol_Cloud_Corp_Earnings_Report.pdf")

file_ids = [file_id]
print(f"\n✅ File uploaded successfully")

### Step 3: Create Vector Store

Create a vector store with the uploaded files:

In [ ]:
print("🔧 Creating vector store...\n")

vector_store = client.vector_stores.create(
    name="parasol_earnings_fy2025",
    file_ids=file_ids,
    #embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    #embedding_dimension=384,
    #provider_id="milvus"
)

print("✅ Vector store created successfully!")
print(f"   ID: {vector_store.id}")
print(f"   Name: {vector_store.name}")
print(f"   Embedding Model: sentence-transformers/all-MiniLM-L6-v2")
print(f"   Dimensions: 384")
print(f"   Provider: Milvus")

## Test Direct Vector Search

Test semantic search by querying the vector store directly:

In [ ]:
print("🔍 Testing vector search...\n")

test_query = "What is the total revenue for FY2025?"
print(f"Query: {test_query}\n")

# Perform semantic search
search_response = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query=test_query,
    max_num_results=3
)

# Display results
print("📊 Search Results:\n")
for i, result in enumerate(search_response.data, 1):
    content = result.content[0].text
    filename = result.filename if hasattr(result, 'filename') else 'Unknown'
    print(f"   [{i}] {filename}")
    print(f"       Content: {content[:200]}...")  # Show first 200 chars
    print()

print("✅ Vector search working correctly!")

## Agentic RAG Function

Define a function that implements the Agentic RAG loop:

1. Search the vector store for relevant chunks
2. Extract context from search results
3. Build a prompt with context + query
4. Call Responses API to generate answer

In [ ]:
def agentic_rag(client, vector_store_id, query, model, max_results=3):
    """
    Perform Agentic RAG: search → extract context → generate answer
    
    Args:
        client: LlamaStackClient instance
        vector_store_id: ID of the vector store to search
        query: User's question
        model: Model ID for inference
        max_results: Number of chunks to retrieve
    
    Returns:
        dict with query, context, answer, and metadata
    """
    # 1. Search vector store for relevant chunks
    search_results = client.vector_stores.search(
        vector_store_id=vector_store_id,
        query=query,
        max_num_results=max_results
    )
    
    # 2. Extract context from search results using traditional loop
    context_chunks = []
    for result in search_results.data:
        text = result.content[0].text
        context_chunks.append(text)
    
    context = "\n\n".join(context_chunks)
    
    # 3. Build prompt with context
    prompt = f"""Use the following context to answer the question.

Context:
{context}

Question: {query}

Answer based only on the context provided above."""
    
    # 4. Call Responses API to generate answer
    response = client.responses.create(
        model=model,
        input=prompt
    )
    
    return {
        "query": query,
        "context": context,
        "answer": response.output_text,
        "num_chunks": len(context_chunks)
    }

print("✅ Agentic RAG function defined")

## Example 1: Revenue Query

Test the RAG system with a financial metrics question:

In [ ]:
print("\n" + "="*60)
print("🚀 Example 1: Revenue Query")
print("="*60 + "\n")

query = "What is the total revenue of Parasol Cloud Corp in FY2025?"
print(f"❓ Query: {query}\n")

# Perform agentic RAG
result = agentic_rag(
    client=client,
    vector_store_id=vector_store.id,
    query=query,
    model=INFERENCE_MODEL,
    max_results=3
)

# Display results
print(f"🔍 Retrieved {result['num_chunks']} relevant chunks\n")
print("📄 Context:")
print("-" * 60)
print(result['context'][:500] + "..." if len(result['context']) > 500 else result['context'])
print("-" * 60)

print("\n💡 Answer:")
print("=" * 60)
print(result['answer'])
print("=" * 60)

## Example 2: Customer Query

Test with a question about top customers:

In [ ]:
print("\n" + "="*60)
print("🚀 Example 2: Customer Query")
print("="*60 + "\n")

query = "Who are the top customers of Parasol Cloud Corp?"
print(f"❓ Query: {query}\n")

# Perform agentic RAG
result = agentic_rag(
    client=client,
    vector_store_id=vector_store.id,
    query=query,
    model=INFERENCE_MODEL,
    max_results=3
)

# Display results
print(f"🔍 Retrieved {result['num_chunks']} relevant chunks\n")
print("📄 Context:")
print("-" * 60)
print(result['context'][:500] + "..." if len(result['context']) > 500 else result['context'])
print("-" * 60)

print("\n💡 Answer:")
print("=" * 60)
print(result['answer'])
print("=" * 60)

## Example 3: Operational Metrics Query

Test with a question about operational performance:

In [ ]:
print("\n" + "="*60)
print("🚀 Example 3: Operational Metrics Query")
print("="*60 + "\n")

query = "What is the SLA compliance rate for Parasol Cloud Corp?"
print(f"❓ Query: {query}\n")

# Perform agentic RAG
result = agentic_rag(
    client=client,
    vector_store_id=vector_store.id,
    query=query,
    model=INFERENCE_MODEL,
    max_results=3
)

# Display results
print(f"🔍 Retrieved {result['num_chunks']} relevant chunks\n")
print("📄 Context:")
print("-" * 60)
print(result['context'][:500] + "..." if len(result['context']) > 500 else result['context'])
print("-" * 60)

print("\n💡 Answer:")
print("=" * 60)
print(result['answer'])
print("=" * 60)

---

## Summary

We have successfully built an Agentic RAG system using **pure Llama Stack primitives and Responses API** with a real-world earnings report PDF!

### What We Demonstrated:

✅ **PDF Download** - Fetched PDF document from URL  
✅ **Vector Store Creation** - Created a vector database with embedded PDF content  
✅ **Document Upload** - Uploaded PDF using the Files API  
✅ **Semantic Search** - Performed vector similarity search on financial data  
✅ **Context Extraction** - Retrieved relevant document chunks from earnings report  
✅ **Response Generation** - Used Responses API for question answering  

### Key Llama Stack Primitives Used:

| API | Purpose | Method |
|-----|---------|--------|
| **Files API** | Upload PDF documents | `client.files.create()` |
| **Vector Stores API** | Create vector database | `client.vector_stores.create()` |
| **Search API** | Semantic search | `client.vector_stores.search()` |
| **Responses API** | Generate answers | `client.responses.create()` |

### Agentic RAG Workflow:

```
1. Download PDF from URL
   ↓
2. Upload to Llama Stack Files API
   ↓
3. Create vector store with PDF embeddings
   ↓
4. User asks question
   ↓
5. vector_stores.search() → Find relevant chunks
   ↓
6. Extract context from search results
   ↓
7. Build prompt with context + query
   ↓
8. responses.create() → Generate answer
   ↓
9. Return final answer with citations
```